# Perceptron

## Student Information

|Name|NetId|
|-|-|
|Zhengchuan Liang|zlian064|

## Introduction

Some libraries should be imported first:

In [1]:
# shared code
import math
import numpy as np
import pandas as pd

Load our dataset into a dataframe:

In [2]:
# shared code
df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')

Get `X` (a matrix of the features values) and `y` (a vector of the labels) from the dataframe:

In [3]:
# shared code
y: pd.Series = df['NObeyesdad'].str.startswith('Obesity').astype(int)
del df['NObeyesdad']
del df['Height']
del df['Weight']
X: pd.DataFrame = df

Accuracy, precision, recall and f1 are our metrics:

In [4]:
# shared code
def evaluate(y_actual, y_pred):
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
    print('accuracy:', accuracy_score(y_actual, y_pred))
    print('precision:', precision_score(y_actual, y_pred))
    print('recall:', recall_score(y_actual, y_pred))
    print('f1:', f1_score(y_actual, y_pred))
    print('confusion matrix:')
    print(confusion_matrix(y_actual, y_pred))

## Preprocessing

Since perceptron only accepts numeric values, I need to map categorical values to numeric ones.
For `MTRANS` feature, I believe the number reflects the amount of exercise.

Then normalize all features using min-max.

In [5]:
def preprocessing_perceptron(X: pd.DataFrame) -> pd.DataFrame:
    freq_map = {'no': 0, 'Sometimes': 1,
                'Frequently': 2, 'Always': 3}

    yes_no_map = {'yes': 1, 'no': 0}

    X = X.copy()
    X['Gender'] = X['Gender'].map({'Male': 1, 'Female': 0})
    X['family_history_with_overweight'] = X['family_history_with_overweight'].map(
        yes_no_map)
    X['FAVC'] = X['FAVC'].map(yes_no_map)
    X['CAEC'] = X['CAEC'].map(freq_map)
    X['SMOKE'] = X['SMOKE'].map(yes_no_map)
    X['SCC'] = X['SCC'].map(yes_no_map)
    X['CALC'] = X['CALC'].map(freq_map)
    X['MTRANS'] = X['MTRANS'].map(
        {'Automobile': 0, 'Motorbike': 0, 'Public_Transportation': 0, 'Walking': 1, 'Bike': 1})

    X = (X-X.min())/(X.max()-X.min())
    return X

## My Implementation

The activation function of my perceptron is a step funcion:

\begin{equation}
F(net)=
\begin{cases}
1 & net>0 \\
0 & net\le 0
\end{cases}
\end{equation}

$$
net=\sum_{i=0}^nw_ix_i=\mathbf{w}^T\mathbf{x}\text{, where }x_0=1
$$

Therefore, the perceptron is:

$$
P(\mathbf{x})=F(\mathbf{w}^T\mathbf{x})
$$

Then we want to minimize the sum of distance between each misclassified sample to the decision boundary.

For a single sample $(\mathbf{x},y)$ where $\mathbf{x}\in\mathbb{R}^n$ and $y\in\mathbb{R}$,
the distance is:

$$
d
=\frac{|\sum_{i=0}^nw_ix_i|}{\sqrt{\sum_{i=1}^nw_i^2}}
=\frac{(P(\mathbf{x})-y)\mathbf{w}^T\mathbf{x}}{||\mathbf{w}||}
$$

We can ignore $||\mathbf{w}||$ (can be proved), then the loss function is:
$$
L(\mathbf{w})=(P(\mathbf{x})-y)\mathbf{w}^T\mathbf{x}
$$

And the gradient is:
$$
\frac{\partial}{\partial w_i}L(\mathbf{w})=(P(\mathbf{x})-y)x_i
$$

$$
\nabla L(\mathbf{w})=(P(\mathbf{x})-y)\mathbf{x}
$$

In my implementation, I use **batch gradient** for all samples instead.

Given features matrix $\mathbf{X}\in\mathbb{R}^{m\times n}$ and label vector $\mathbf{y}\in\mathbb{R}^{m}$, the steps to train a perceptron are as follows:

Initialize weights $\mathbf{w}$ with a random vector.

Predict the labels $\mathbf{y}_{pred}$ using the perceptron.

The sum of loss is:
$$
L(\mathbf{w})
=\sum_{(\mathbf{x},y)}(P(\mathbf{x})-y)\mathbf{w}^T\mathbf{x}
=(\mathbf{y}_{pred}-\mathbf{y})^T\mathbf{X}\mathbf{w}
$$

The average gradient is:
$$
\nabla L(\mathbf{w})^T=\frac{1}{m}(\mathbf{y}_{pred}-\mathbf{y})^T\mathbf{X}
$$

Update the weights:
$$
\mathbf{w}\leftarrow \mathbf{w}-\eta\nabla L(\mathbf{w})
$$

Repeat the above steps for 200 times.

In [6]:
def add_ones_column(X: np.ndarray) -> np.ndarray:
    """ add a column of ones to the matrix
    """
    X0 = np.ones((X.shape[0], 1))
    return np.hstack((X0, X))


class MyPerceptron:
    def __init__(self, lr: float) -> None:
        self.lr: float = lr  # learning rate

    def fit(self, X: np.ndarray, y: np.ndarray, seed: int = 0) -> None:
        assert X.ndim == 2
        assert y.ndim == 1
        assert X.shape[0] == y.shape[0]

        X = add_ones_column(X)
        m = X.shape[0]  # number of samples
        n = X.shape[1]  # number of features

        # weights
        rng = np.random.default_rng(seed=seed)
        init_W = rng.random(n)
        init_W /= np.linalg.norm(init_W)  # initial weights are a random unit vector
        self.W: np.ndarray = init_W

        for epoch in range(200):
            # predictions for all samples. vector of size m
            y_pred = np.where(np.dot(X, self.W) > 0, 1, 0)

            # (average) gradient for all samples. vector of size n
            gradient = (y_pred - y) @ X / m

            # loss for all samples
            loss = (y_pred - y) @ (X @ self.W)

            # update weights
            self.W -= self.lr * gradient

    def predict(self, X: np.ndarray) -> np.ndarray:
        X = add_ones_column(X)
        return np.where(np.dot(X, self.W) > 0, 1, 0)

## Comparison

To reduce the randomness of train_test_split, I split the dataset with 50 different random_states. For each split I train the off-the-shelf implementation and mine using the training set and predict the labels for the test set. I concatenate the $y_{actual}$ and $y_{pred}$ of each split and use them to compute the metrics.

My implementation performs better than off-the-shelf on f1 score. This may result from batch gradient descent, whereas the off-the-shelf uses stochastic gradient descent.

I also compute the cosine similarity between the weights of off-the-shelf perceptron and mine. The result shows that the weights of my implementation are very similar to that of off-the-shelf. It is much higher than the average similarity of two random 15-dimensional vectors (around 0.756). This demonstrates that my implementation is correct.

In [7]:
def cosine_sim(x, y):
    """ cosine similarity
    """
    return x @ y / (np.linalg.norm(x) * np.linalg.norm(y))


def run_perceptron(X: pd.DataFrame, y: pd.Series):
    from sklearn.linear_model import Perceptron
    from sklearn.model_selection import train_test_split

    X = preprocessing_perceptron(X)
    X = X.to_numpy()
    y = y.to_numpy()

    y_test_all = np.empty(shape=(0,))
    y_pred_shelf_all = np.empty(shape=(0,))
    y_pred_my_all = np.empty(shape=(0,))
    cosine_all = np.empty(shape=(0,))
    for r in range(50):
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.1, random_state=r, stratify=y)
        y_test_all = np.append(y_test_all, y_test)

        # off-the-shelf library
        shelf = Perceptron(tol=1e-3, random_state=r)
        shelf.fit(X_train, y_train)  # train using training set
        y_pred_shelf = shelf.predict(X_test)  # predict for test set
        y_pred_shelf_all = np.append(y_pred_shelf_all, y_pred_shelf)
        W_shelf = np.append(shelf.intercept_, shelf.coef_.flatten())  # weights of library implementation

        # my implementation
        my = MyPerceptron(1)
        my.fit(X_train, y_train, seed=r)  # train using training set
        y_pred_my = my.predict(X_test)  # predict for test set
        y_pred_my_all = np.append(y_pred_my_all, y_pred_my)
        W_my = my.W  # weights of my implementation

        # cosine similarity
        cosine = cosine_sim(W_shelf, W_my)
        cosine_all = np.append(cosine_all, cosine)

    print('[+] off-the-shelf implementation:')
    evaluate(y_test_all, y_pred_shelf_all)
    print()

    print('[+] my implementation:')
    evaluate(y_test_all, y_pred_my_all)
    print()

    # average cosine similarity
    # between the weights of library and my implementation
    print("[+] average cosine similarity:", cosine_all.mean())

In [8]:
run_perceptron(X.copy(), y.copy())

[+] off-the-shelf implementation:
accuracy: 0.6788679245283019
precision: 0.6497797356828194
recall: 0.6622448979591836
f1: 0.6559531028906408
confusion matrix:
[[3951 1749]
 [1655 3245]]

[+] my implementation:
accuracy: 0.6800943396226415
precision: 0.5956880152187698
recall: 0.9585714285714285
f1: 0.7347673054360578
confusion matrix:
[[2512 3188]
 [ 203 4697]]

[+] average cosine similarity: 0.9453590450484168
